In [1]:
pip install langchain langchain-community langchain-huggingface langchain-chroma langchain-ollama pypdf

In [6]:
import os
from langchain_community.document_loaders import PyPDFLoader
# folder_path = "/content/sample_data/sample_docs"

def load_docs(folder_path:str):
  if not os.path.exists(folder_path):
    raise FileNotFoundError(f"Folder {folder_path} does not exist")

  documents = []
  for file in os.listdir(folder_path):
    if file.endswith(".pdf"):
      file_path = os.path.join(folder_path,file)
      print(f"Loading {file_path}")
      try:
        loader = PyPDFLoader(file_path)
        documents.extend(loader.load())
      except Exception as e:
        print(f"Error loading {file}: {e}")

  return documents

2. Chunking

In [7]:
#We use RecursiveCharacterTextSplitter, which is a smart tool.
#It tries to split text by paragraphs first, then by sentences, so related text stays together

from langchain_text_splitters import RecursiveCharacterTextSplitter

def split_text(documents):
  splitter = RecursiveCharacterTextSplitter(
      chunk_size=1000,
      chunk_overlap=200,
      # length_function=len
  )

  chunks = splitter.split_documents(documents)
  print(f"Created {len(chunks)} chunks")
  return chunks

3. Embeddings


In [8]:
#we need to turn our text chunks into lists of numbers, called vectors or embeddings.
#We will use the 'all-MiniLM-L6-v2' open source model

from langchain_community.embeddings import HuggingFaceEmbeddings
import sentence_transformers

embedding_function = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)

Loading weights:   0%|          | 0/103 [00:00<?, ?it/s]

BertModel LOAD REPORT from: sentence-transformers/all-MiniLM-L6-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


4. Vector Store

In [15]:
"""we need somewhere to store these numbers for fast searching.
A regular SQL database isn’t good for this, so we’ll use a Vector Database. We’ll use Chroma"""

from langchain_chroma import Chroma

def new_vector_store(chunks):
  vector_store = Chroma.from_documents(
      documents = chunks,
      embedding = embedding_function,
      persist_directory="./chroma_db",
      collection_name="Sample_rag_docs"
  )

  return vector_store


5. LLM

In [13]:
#This function links the user, the database, and the LLM

from langchain_ollama import ChatOllama
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

def format_docs(docs):
  return "\n\n".join(doc.page_content for doc in docs)


def query_rag_system(query_text,vector_store):
  llm = ChatOllama(model="llama3")
  retriever = vector_store.as_retriever(search_kwargs = {'k':3})
  prompt= ChatPromptTemplate.from_template(
      """
        You are a helpful assistant.
        Answer ONLY using the context below.
        If the answer is not present, say "I don't know."

        Context:
        {context}

        Question:
        {question}
        """
  )

  chain = (
        {
            "context": retriever | format_docs,
            "question": RunnablePassthrough(),
        }
        | prompt
        | llm
        | StrOutputParser()
    )
  return chain.invoke(query_text)

In [20]:
def main():
  folder_path = "/content/sample_data/sample_docs"

  if not os.path.exists("./chroma_db"):
    print("No vector DB found, creating one")
    docs = load_docs(folder_path)
    chunks = split_text(docs)
    vector_store = new_vector_store(chunks)

    print("Vector DB created")
  else:
    print("Loading vector DB")
    vector_store = Chroma(
        persist_directory="./chroma_db",
        embedding_function=embedding_function,
        collection_name="Sample_rag_docs"
    )

  while True:
    query = input("Ask a questions : ")
    if query.lower() == "exit":
      break

    print("Thinking...")
    answer = query_rag_system(query,vector_store)
    print("\n Answer:  \n",answer)

if __name__ == "__main__":
  main()



Loading vector DB
Ask a questions : nlp
Thinking...


ConnectError: [Errno 111] Connection refused